# Mastermind game - Genetic Algorithms (DEAP solution)
In this notebook we solve the mastermind game using genetic algorithms

**NOTE FOR STUDENTS**: Before checking this solution, please make sure to attempt the exercise on your own to maximize your learning experience.

In [1]:
# Code formatting in jupyter notebooks with black
import jupyter_black

jupyter_black.load()

In [2]:
# import the game module
import sentence_mastermind

## Create an instance of the game

In [4]:
# Create the hidden sentence
mastermind = sentence_mastermind.SentenceMastermind("Hello, World")
mastermind.set_sentence_from_file()

# we get some information to create the individual
individual_length = mastermind.get_sentence_length()
possible_characters = mastermind.get_unique_characters_in_file()

In [5]:
## Genetic Algorithm using deap library
# import the necessary libraries
from deap import base
from deap import creator
from deap import tools

## GA

### Creator

In [6]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # maximize the fitness
creator.create(
    "Individual", list, fitness=creator.FitnessMax
)  # individual is a list with a fitness attribute

### Toolbox

In [7]:
# we define a function to select a random character from a string
import random


def select_random_char_from_string(my_string=possible_characters):
    selected_char = random.choice(my_string)
    # print("selected_char", selected_char)
    return selected_char


toolbox = base.Toolbox()
# Attribute generator
toolbox.register("attr_char", select_random_char_from_string)
# Structure initializers
toolbox.register(
    "individual",
    tools.initRepeat,
    creator.Individual,
    toolbox.attr_char,
    individual_length,
)  # our individual is a list of characters, our "guess"
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

### Evaluation function

In [8]:
def evalMastermind(individual):
    guess = individual  # renamed for clarity
    correct_position, correct_character = mastermind.check_guess(guess)

    # as simple fitness function, we give a weight of 3 to the correct position and 0.5 to the correct character
    fitness_value = correct_position * 3 + correct_character * 0.5

    # we normalize the fitness value
    max_fitness = (
        3 * individual_length
    )  # this means that all characters are in the correct position
    fitness_value = fitness_value / max_fitness
    return (fitness_value,)

### The Genetic Operators

In [9]:
toolbox.register("evaluate", evalMastermind)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05) # 0.05 is the probability of mutating each character
toolbox.register("select", tools.selTournament, tournsize=3)

### Evolving the Population

The code below has been adapted from the examples on [deap documentation](https://deap.readthedocs.io/en/master/examples/ga_onemax.html).

In [10]:
import random
import matplotlib.pyplot as plt


def main():
    random.seed(42)
    # Creating the Population
    pop = toolbox.population(n=500)

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    # CXPB  is the probability with which two individuals are crossed
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = 0.5, 0.2

    # List to store the average fitness per generation
    avg_fitness_per_gen = []

    # Performing the Evolution
    # Extracting all the fitnesses of
    fits = [ind.fitness.values[0] for ind in pop]
    # Variable keeping track of the number of generations
    g = 0

    # Begin the evolution
    while (
        max(fits) < 1 and g < 2000
    ):  # if 1, it means that we have found the hidden sentence
        # A new generation
        g = g + 1
        print(f"-- Generation {g} --")
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
        # Apply crossover and mutation on the offspring
        # NOTE: we select two individuals at a time (1 with even index, 1 with odd index), mate them and then mutate them
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values  # fitness values of the children are invalidated
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values # fitness values of the mutants are invalidated

        # Evaluate the individuals with an invalid fitness
        # Note: We are reevaluating the fitness of the individuals that have been mutated and crossed over
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        pop[:] = offspring

        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]

        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x * x for x in fits)
        std = abs(sum2 / length - mean**2) ** 0.5

        print(f"  Min {min(fits)}")
        print(f"  Max {max(fits)}")
        print(f"  Avg {mean}")
        print(f"  Std {std}")

        # Append the average fitness of this generation to the list
        avg_fitness_per_gen.append(mean)

    print("-- End of evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    best_ind_str = "".join(best_ind)
    print(
        f"The best individual is: {best_ind_str} \nwith a fitness score of: {best_ind.fitness.values[0]}, after {g} generations."
    )
    print(f"The hidden sentence is: {mastermind.get_sentence()}")

    # Plotting the average fitness per generation
    plt.plot(avg_fitness_per_gen)
    plt.xlabel("Generation")
    plt.ylabel("Average Fitness")
    plt.title("Average Fitness per Generation")
    plt.show()

In [ ]:
if __name__ == "__main__":
    main()